In [ ]:
!wget "https://api.zindi.africa/v1/competitions/artificial-intelligence-challenge-advanced/files/license_plates_detection_train.zip?auth_token=zus.v1.QzSmY26.e4du7TzJdi1eZbwSRZPEsfrGY3JdvW" -O "license_plates_detection_train.zip"
!wget "https://api.zindi.africa/v1/competitions/artificial-intelligence-challenge-advanced/files/test.zip?auth_token=zus.v1.QzSmY26.e4du7TzJdi1eZbwSRZPEsfrGY3JdvW" -O "test.zip"
!wget "https://api.zindi.africa/v1/competitions/artificial-intelligence-challenge-advanced/files/license_plates_recognition_train.zip?auth_token=zus.v1.QzSmY26.e4du7TzJdi1eZbwSRZPEsfrGY3JdvW" -O "license_plates_recognition_train.zip"
!wget "https://api.zindi.africa/v1/competitions/artificial-intelligence-challenge-advanced/files/license_plates_detection_train.csv?auth_token=zus.v1.QzSmY26.e4du7TzJdi1eZbwSRZPEsfrGY3JdvW" -O "license_plates_detection_train.csv"
!wget "https://api.zindi.africa/v1/competitions/artificial-intelligence-challenge-advanced/files/license_plates_recognition_train.csv?auth_token=zus.v1.QzSmY26.e4du7TzJdi1eZbwSRZPEsfrGY3JdvW" -O "license_plates_recognition_train.csv"

--2023-12-07 15:43:45--  https://api.zindi.africa/v1/competitions/artificial-intelligence-challenge-advanced/files/license_plates_detection_train.zip?auth_token=zus.v1.QzSmY26.e4du7TzJdi1eZbwSRZPEsfrGY3JdvW
Resolving api.zindi.africa (api.zindi.africa)... 18.135.135.166, 13.43.222.50, 18.171.65.214
Connecting to api.zindi.africa (api.zindi.africa)|18.135.135.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘license_plates_detection_train.zip’

license_plates_dete     [      <=>           ]  92.67M  11.2MB/s    in 9.3s    

2023-12-07 15:43:56 (9.96 MB/s) - ‘license_plates_detection_train.zip’ saved [97176102]

--2023-12-07 15:43:56--  https://api.zindi.africa/v1/competitions/artificial-intelligence-challenge-advanced/files/test.zip?auth_token=zus.v1.QzSmY26.e4du7TzJdi1eZbwSRZPEsfrGY3JdvW
Resolving api.zindi.africa (api.zindi.africa)... 18.135.135.166, 13.43.222.50, 18.171.65.214
Connecting to api.zindi.afric

In [ ]:
x = !ls *.zip
x = x[0].split()
for g in x:
  !unzip $g

Archive:  license_plates_detection_train.zip
   creating: license_plates_detection_train/
  inflating: license_plates_detection_train/1.jpg  
  inflating: license_plates_detection_train/2.jpg  
  inflating: license_plates_detection_train/3.jpg  
  inflating: license_plates_detection_train/4.jpg  
  inflating: license_plates_detection_train/5.jpg  
  inflating: license_plates_detection_train/6.jpg  
  inflating: license_plates_detection_train/7.jpg  
  inflating: license_plates_detection_train/8.jpg  
  inflating: license_plates_detection_train/9.jpg  
  inflating: license_plates_detection_train/10.jpg  
  inflating: license_plates_detection_train/11.jpg  
  inflating: license_plates_detection_train/12.jpg  
  inflating: license_plates_detection_train/13.jpg  
  inflating: license_plates_detection_train/14.jpg  
  inflating: license_plates_detection_train/15.jpg  
  inflating: license_plates_detection_train/16.jpg  
  inflating: license_plates_detection_train/17.jpg  
  inflating: licen

In [ ]:
!pip install ultralytics
!pip install tqdm --upgrade




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.0/654.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
import ultralytics
import matplotlib.pyplot as plt
import os
import shutil
import rimport ultralytics
import yaml
import pandas as pd
import torch
import numpy as np
import re
import os

from torch.utils.data import Dataset
from ultralytics import YOLO
from tqdm import tqdm
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from PIL import Image
from transformers import TrOCRProcessor
from transformers import VisionEncoderDecoderModel
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from google.colab.patches import cv2_imshow
from scipy.ndimage import interpolation as inter
from IPython.display import display


ultralytics.checks()

Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.1/78.2 GB disk)


In [ ]:
!rm /content/test
!rm /content/train

rm: cannot remove '/content/test': No such file or directory
rm: cannot remove '/content/train': No such file or directory


In [ ]:
def pascal_voc_to_yolo(x1, y1, x2, y2, image_w, image_h):
    return [((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]

In [ ]:
df_d = pd.read_csv("/content/license_plates_detection_train.csv")
df_d['class'] = [0 for i in range(len(df_d))]
df_d = df_d[['img_id','class', 'xmin', 'ymin','xmax','ymax']]

In [ ]:
train_df, test_df = train_test_split(df_d, test_size=0.2)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:
!mkdir "train"
!mkdir 'test'

!mkdir "train/images"
!mkdir 'test/images'

!mkdir "train/labels"
!mkdir 'test/labels'

In [ ]:
img_path = "/content/license_plates_detection_train/"


for i in range(len(train_df)):
  path = img_path+train_df.iloc[i,0]
  shutil.move(path,"/content/train/images")

for i in range(len(test_df)):
  path = img_path+test_df.iloc[i,0]
  shutil.move(path,"/content/test/images")


In [ ]:
for i in range(len(train_df)):
  img_path = "/content/train/images/"
  path = img_path+train_df.iloc[i,0]
  img = Image.open(path)

  h,w = img.height,img.width

  b = pascal_voc_to_yolo(train_df.iloc[i,2],train_df.iloc[i,3],train_df.iloc[i,4],train_df.iloc[i,5],w,h)

  train_df.iloc[i,2] = b[0]
  train_df.iloc[i,3] = b[1]

  train_df.iloc[i,4] = b[2]
  train_df.iloc[i,5] = b[3]


for i in range(len(test_df)):
  img_path = "/content/test/images/"
  path = img_path+test_df.iloc[i,0]
  img = Image.open(path)

  h,w = img.height,img.width

  b = pascal_voc_to_yolo(test_df.iloc[i,2],test_df.iloc[i,3],test_df.iloc[i,4],test_df.iloc[i,5],w,h)

  test_df.iloc[i,2] = b[0]
  test_df.iloc[i,3] = b[1]

  test_df.iloc[i,4] = b[2]
  test_df.iloc[i,5] = b[3]




In [ ]:
for i in range(len(train_df)):
  data = train_df.iloc[i,1:].values
  name = train_df.iloc[i,0]
  path = '/content/train/labels/'+name[:-4]+".txt"
  with open(path, "w") as txt_file:
      for line in data:
          txt_file.write("".join(" "+str(line)) )


for i in range(len(test_df)):
  data = test_df.iloc[i,1:].values
  name = test_df.iloc[i,0]
  path = '/content/test/labels/'+name[:-4]+".txt"
  with open(path, "w") as txt_file:
      for line in data:
          txt_file.write("".join(" "+str(line)) )

In [ ]:
ultralytics.checks()

Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.1/78.2 GB disk)


In [ ]:
plates_yaml = dict(
    train ="/content/train",
    val ='/content/test',
    test='',
    nc = 1,
    names =['plates']
)


In [ ]:
with open('plates.yaml', 'w') as outfile:
    yaml.dump(plates_yaml, outfile, default_flow_style=True)

In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data=/content/plates.yaml epochs=40 imgsz=640 batch=16 project=./detection name=plates


100% 6.23M/6.23M [00:00<00:00, 74.7MB/s]
Ultralytics YOLOv8.0.222 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/plates.yaml, epochs=40, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=./detection, name=plates, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show

In [ ]:
tpaths2=[]
for dirname, _, filenames in os.walk('/content/detection/plates'):
    for filename in filenames:
        if filename[-4:]=='.png' or filename[-4:]=='.jpg':
            tpaths2+=[(os.path.join(dirname, filename))]
tpaths2=sorted(tpaths2)
print(tpaths2[0])

/content/detection/plates/F1_curve.png


In [ ]:
for path in tpaths2:
    image = Image.open(path)
    image=np.array(image)
    plt.figure(figsize=(20,10))
    plt.imshow(image)
    plt.show()

In [ ]:
best_path0='/content/detection/plates/weights/best.pt'

source0='/content/test_private'

In [ ]:
ppaths=[]
for dirname, _, filenames in os.walk(source0):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            ppaths+=[(os.path.join(dirname, filename))]
ppaths=sorted(ppaths)
print(ppaths)

In [ ]:
model2 = YOLO(best_path0)

In [ ]:
results = model2.predict(source0,conf=0.4)
print(len(results))

In [ ]:
results[0].boxes

In [ ]:
PBOX=pd.DataFrame(columns=range(6))
for i in range(len(results)):
    arri=pd.DataFrame(results[i].boxes.xyxy.tolist()).astype(float)
    path=ppaths[i]
    file=path.split('/')[-1]
    arri=arri.assign(file=file)
    arri=arri.assign(i=i)
    PBOX=pd.concat([PBOX,arri],axis=0)
PBOX.columns=['x','y','x2','y2','confidence','class','file','i',]
display(PBOX)

In [ ]:
PBOX['class'] = [0 for i in range(len(PBOX))]
PBOX['confidence'] = [1 for i in range(len(PBOX))]

In [ ]:
def draw_box2(n0):

    ipath=ppaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath.split('/')[-1]

    if PBOX[PBOX['file']==file] is not None:
        box=PBOX[PBOX['file']==file]
        box=box.reset_index(drop=True)
        #display(box)

        for i in range(len(box)):
            label=box.iloc[i,5]
            x=int(box.iloc[i,0])
            y=int(box.iloc[i,1])
            x2=int(box.iloc[i,2])
            y2=int(box.iloc[i,3])
            #print(label,x,y,x2,y2)


            cv2.rectangle(image,(x,y),(x2,y2),(0,255,0),5) #green

    #plt.imshow(image)
    #plt.show()

    return image

In [ ]:
images2=[]
for i in tqdm(range(len(ppaths))):
    images2+=[draw_box2(i)]

In [ ]:
def draw_box(n0):

    ipath=ipaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath[0:-4].split('/')[-1]

    if BOX2[BOX2[5]==file] is not None:
        box=BOX2[BOX2[5]==file]
        box=box.reset_index(drop=True)

        for i in range(len(box)):
            label=box.iloc[i,0]
            x=box.iloc[i,1]
            y=box.iloc[i,2]
            w=box.iloc[i,3]
            h=box.iloc[i,4]
            x0=((x-w/2)*W).astype(int)
            y0=((y-h/2)*H).astype(int)
            x1=((x+w/2)*W).astype(int)
            y1=((y+h/2)*H).astype(int)
            if label=='player':
                cv2.rectangle(image,(x0,y0),(x1,y1),(0,255,0),5) #green
            elif label=='football':
                cv2.rectangle(image,(x0,y0),(x1,y1),(0,0,255),5) #blue

    #plt.imshow(image)
    #plt.show()

    return image

In [ ]:
from matplotlib import rc,animation
rc('animation', html='jshtml')
def create_animation(ims):
    fig=plt.figure(figsize=(10,6))
    #plt.axis('off')
    im=plt.imshow(cv2.cvtColor(ims[0],cv2.COLOR_BGR2RGB))
    plt.close()
    def animate_func(i):
        im.set_array(cv2.cvtColor(ims[i],cv2.COLOR_BGR2RGB))
        return [im]
    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000//2)

In [ ]:
create_animation(images2)

In [ ]:
PBOX.to_csv("roi.csv")

# Recogntion



In [ ]:
!pip install -q transformers
!pip install sentencepiece
!pip install datasets
!pip install jiwer

In [ ]:
df = pd.read_csv("/content/license_plates_recognition_train.csv")

In [ ]:

train_df, test_df = train_test_split(df, test_size=0.2)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

In [ ]:


class Plates(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text,
                                          padding="max_length",
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

In [ ]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")

train_dataset = Plates_Rec(root_dir='/content/license_plates_recognition_train/',
                           df=train_df,
                           processor=processor)

eval_dataset = Plates_Rec(root_dir='/content/license_plates_recognition_train/',
                           df=test_df,
                           processor=processor)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [ ]:
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 720
Number of validation examples: 180


In [ ]:
encoding = train_dataset[0]
for k,v in encoding.items():
  print(k, v.shape)

pixel_values torch.Size([3, 384, 384])
labels torch.Size([128])


In [ ]:
image = Image.open(train_dataset.root_dir + train_df['file_name'][0]).convert("RGB")
image

In [ ]:
labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)
print(label_str)

90T1397


In [ ]:
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

In [ ]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [ ]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    fp16=True,
    output_dir="./",
    logging_steps=2,
    save_steps=200,
    eval_steps=200,
    num_train_epochs=7,
)

In [ ]:
model2 = VisionEncoderDecoderModel.from_pretrained("/content/checkpoint-600")
for file_name in test_df['file_name']:
  image = Image.open(train_dataset.root_dir + file_name).convert("RGB")
  display(image)

  generated_ids = model2.generate(processor(image, return_tensors="pt").pixel_values)
  print(processor.batch_decode(generated_ids, skip_special_tokens=True)[0])

In [ ]:
def predict(img_path,x1,y1,x2,y2):
  img = cv2.imread(img_path)
  roi = img[y1:y2,x1:x2]
  cv2_imshow(roi)
  generated_ids = model2.generate(processor(roi, return_tensors="pt").pixel_values)
  predicted_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
  print("Predicted Text:", predicted_text)
  return predicted_text

def rep(x):
  x = re.sub("[^0-9]","",x)
  return x

def rep2(x):
  x = re.sub(".jpg","",x)
  return x

In [ ]:
dr = pd.read_csv("roi.csv")

dr.drop(['Unnamed: 0','confidence','class','i'],axis=1,inplace=True)

path = "/content/test_private/"

names = dr['file'].drop_duplicates().values.tolist()

preds = []
for i in names:
  print(i)
  x1,y1,x2,y2 = dr[dr['file'].str.contains(i)].values.tolist()[0][:-1]
  g = predict(path+i,int(x1),int(y1),int(x2),int(y2))
  preds.append(g)


In [ ]:
pred = pd.DataFrame(preds,columns=["preds"])

pred['images'] = names

pred['preds'] = pred['preds'].apply(lambda x: rep(x))

pred['images'] = pred['images'].apply(lambda x: rep2(x))

pred['images'] = pred['images'].apply(lambda x: "img_"+x)

pred['preds'] = pred['preds'].apply(lambda x : x.rjust(7, "-"))

In [ ]:
x = pred['preds'].values

id = [pred['images'][x]+"_"+str(i+1)  for x in range(210) for i in range(7)]

samp = pd.read_csv("/content/SampleSubmission.csv")

m = np.zeros((1470,10))
l = 0
for i in range(210):
  for k in range(7):
    tt = x[i][k]
    try :
      tt = int(tt)
    except:
      pass
    if type(tt) == int:
      m[l][tt] = 1
    else:
      pass
    if l <1470:
      l += 1

b = pd.DataFrame(m,columns=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
b['id'] = id
er = b[['id','0', '1', '2', '3', '4', '5', '6', '7', '8', '9' ]]
er.to_csv("subm.csv",index=False)